# Вопрос 1

In [349]:
import pandas as pd
import numpy as np
import datetime as dt

Создадим для примера собственные DataFrames, соответствующие описанию DataFrames, которые должны быть загружены из БД.

In [350]:
d = np.array([['2018-09-12 12:00:00','1','1'], ['2018-10-12 12:00:00', '2', '2'], ['2019-06-03 12:00:00', '5', '3'], ['2019-06-10 12:00:00', '3', '4'],['2019-04-22 12:00:00', 1, '5'],['2019-04-04 12:00:00', '4', '6'],['2019-06-03 12:00:00', '2', '7'],['2019-06-01 12:00:00', '5', '8'],['2019-06-05 12:00:00', '1' , '9'],['2019-06-11 12:00:00', '5', '10']])

In [351]:
orders = pd.DataFrame(columns= ['DateTime', 'CustomerId', 'OrderId'], data = d)

In [352]:
dor = np.array([['1','2',300],['1','5',200],['2','4',1000],['3','8',150],['3','2',300],['4','5',200],['5','7',450],['5','9',180],['5','2',300],['6','1',1300],['7','6',35],['8','5',200],['8','4',1000],['8','7',450],['9','11',440],['10','11',440]])

In [353]:
orderLines = pd.DataFrame(columns = ['OrderId', 'ProductId', 'Price'], data = dor)

In [354]:
#оставили только заказы, сделанные меньше месяца назад и привели определенные данные к нужным типам
df_joined = orders.set_index('OrderId').join(orderLines.set_index('OrderId'), sort = True, how = 'inner').reset_index()
df_joined['Price'] = pd.to_numeric(df_joined['Price'])
df_joined['DateTime'] = pd.to_datetime(df_joined['DateTime'])
df_joined = df_joined[(now - df_joined['DateTime'])/np.timedelta64(1, 'M')<=1]

In [355]:
max_num = int(df_joined.groupby('ProductId').agg({'OrderId':'size'}).rename(columns={'OrderId':'count'}).sort_values('count', ascending=False).agg({'count':'max'}))
max_num
#смотрим, в скольких заказах встречаются самые популярные продукты 

2

In [356]:
# id популярных продуктов и суммарные выручка по каждому из них
pop_pr = df_joined.groupby('ProductId')[['Price','OrderId']].agg({'OrderId':'size','Price':'sum'})[df_joined.groupby('ProductId').agg({'CustomerId':'size'})['CustomerId']== max_num].reset_index()[['ProductId','Price']].rename(columns={'Price':'SumProductRevenue'})
pop_pr


,ProductId,SumProductRevenue
0,11,880
1,5,400


In [362]:
#создадим DataFrame с средними чеками заказов, где есть определенный продукт для этого сделаем пункты 1)-3)
#1) в каких заказах есть поп продукты
ordersWithPopPr = df_joined.set_index('ProductId').join(pop_pr.set_index('ProductId'),how='inner')['OrderId'].reset_index()
ordersWithPopPr

,ProductId,OrderId
0,11,10
1,11,9
2,5,4
3,5,8


In [359]:
#2)сумма каждого заказа
orderSumRev = df_joined.groupby('OrderId').agg({'Price':'sum'}).reset_index()
orderSumRev

,OrderId,Price
0,10,440
1,3,450
2,4,200
3,7,35
4,8,1650
5,9,440


In [360]:
#3)сумма каждого заказа, где есть поп продукты
orderSumRevWithPopPr = ordersWithPopPr.set_index('OrderId').join(orderSumRev.set_index('OrderId'),how = 'inner').reset_index()
#Итог пунктов 1)-3) получили средние суммы заказов, где есть популярные продукты
meanBill = orderSumRevWithPopPr.groupby('ProductId').agg({'Price':'mean'}).rename(columns = {'Price':'MeanBill'}).reset_index()

### Отчет, который демонстрирует популярные продукты (PopularProductId), суммарную выручку по каждому такому продукту (SumProductRevenue) и средний чек заказов, в которых есть такие продукты (MeanBill). За последний месяц:

In [367]:
report = pop_pr.set_index('ProductId').join(meanBill.set_index('ProductId'),how = 'inner').reset_index().rename(columns={'ProductId':'PopularProductId'})
report

,PopularProductId,SumProductRevenue,MeanBill
0,11,880,440
1,5,400,925


# Покрытие отчета тестами

Не совсем понимаю, что я значит покрыть отчет тестами. Предполагаю, это значит проверить верность отчета различными способами.

1) Проверим, правильно ли алгорит выбрал популярные продукты 

In [373]:
#создаем dataFrame, которые показывает, в скольких заказах есть каждый продукт в реальности
df_count_or =  df_joined.groupby('ProductId').agg({'OrderId':'size'}).rename(columns={'OrderId':'count'}).sort_values('count', ascending=False)
#смотрим, какое количество заказов соответствует самым популярным продуктам в реальности
max_num_for_test = int(df_count_or.agg({'count':'max'}))

1.1 Проверим совпадает ли количество самых популярных продуктов из отчета с количеством реальных самых популярных продуктов

In [397]:
if report.shape[0] == max_num_for_test:
    print('Test passed without errors')
elif report.shape[0] < max_num_for_test:
    print('The report has less popular products than there are really is')
elif report.shape[0] > max_num_for_test:
    print('The report has more popular products than there are really is')

Test passed without errors


Если тест 1.1 выполнен успешно (Test passed without errors), то есть смысл делать 1.2. Иначе - нет.

1.2 Проверим, совпадают ли самые популярные продукты из теста с реальными популярными продуктами.

In [400]:
#Получаем самые популярные значения в действительности
pop_pr_in_reality = df_count_or[df_count_or['count'] == max_num_for_test].reset_index()['ProductId']

In [401]:
if(report['PopularProductId'] == pop_pr_in_reality).all():
    print('Test passed without errors')
elif (report['PopularProductId'] == pop_pr_in_reality).any() and not (report['PopularProductId'] == pop_pr_in_reality).all():
    print('Some popular products from report does not match the real ones')
elif not (report['PopularProductId'] == pop_pr_in_reality).any():
    print('Test failed')
    

Test passed without errors


2) Проверим, правильно ли в отчете выведены суммарные выручки по каждому из продуктов (SumProductRevenue). Данный тест не зависит от результата тестов 1.1 и 1.2

In [446]:
#Список суммарных выручек от каждого продукта в реальности
pop_pr_sum_rev_real = df_joined.groupby('ProductId')[['Price','OrderId']].agg({'OrderId':'size','Price':'sum'}).rename(columns={'Price':'SumProductRevenue'}).reset_index()
pop_pr_sum_rev_real

,ProductId,OrderId,SumProductRevenue
0,11,2,880
1,2,1,300
2,4,1,1000
3,5,2,400
4,6,1,35
5,7,1,450
6,8,1,150


In [447]:
#список продуктов в отчете
id_report = np.array(report['PopularProductId'])

In [479]:
#Делаем проверку. Смотрим какое значение общей выручки имеют популярные продукты из отчета в реальнсти и сравниваем данные показатели со значениями в отчете
p, m = 0, 0
for i in range(len(id_report)):
    line = pop_pr_sum_rev_real[pop_pr_sum_rev_real['ProductId'] == id_report[i]]
    if line[['SumProductRevenue']].iloc[0][0] == report[report['PopularProductId'] == id_report[i]][['SumProductRevenue']].iloc[0][0]:
        p += 1
    else:
        m += 1
print('Quantity of matched total revenue of each product:', p)        
print('Quantity of mismatched total revenue of each product:', m)
if m == 0:
    print('Test passed without errors')
else:
    print('Test passed with errors')

Quantity of matched total revenue of each product: 2
Quantity of mismatched total revenue of each product: 0
Test passed without errors


3) Проверим, правильно ли в отчете выведены средние чеки заказов в которых есть популярные продукты. Данный тест не зависит от результата тестов 1.1, 1.2, 2

In [487]:
# поп в реальности не нужны! нужны все товары с их ср стоим заказ


In [505]:
# таблица соотношения продуктов и заказов в реальности
pr_or = df_joined[['ProductId','OrderId']]

In [513]:
#посчитали сумму каждого заказа в реальности
summ_ord = df_joined.groupby('OrderId').agg({'Price':'sum'}).rename(columns={'Price':'Bill'}).reset_index()
summ_ord

,OrderId,Bill
0,10,440
1,3,450
2,4,200
3,7,35
4,8,1650
5,9,440


In [524]:
bill_pr_or_real = pr_or.set_index('OrderId').join(summ_ord.set_index('OrderId'),how = 'inner').reset_index()
#получение реального среднего значения чеков в котором есть каждый продукт 
meanBill_real = bill_pr_or_real.groupby('ProductId').agg({'Bill':'mean'}).rename(columns = {'Bill':'MeanBill'}).reset_index()
meanBill_real

,ProductId,MeanBill
0,11,440
1,2,450
2,4,1650
3,5,925
4,6,35
5,7,1650
6,8,450


In [526]:
#сравним для популярных продуктов из отчета значения ср чека в отчете и в действительности. id_report - из предыдущего теста
p, m = 0, 0
for i in range(len(id_report)):
    line = meanBill_real[meanBill_real['ProductId'] == id_report[i]]
    if line[['MeanBill']].iloc[0][0] == report[report['PopularProductId'] == id_report[i]][['MeanBill']].iloc[0][0]:
        p += 1
    else:
        m += 1
print('Quantity of matched mean bills where the popular product is:', p)        
print('Quantity of mismatched mean bills where the popular product is:', m)
if m == 0:
    print('Test passed without errors')
else:
    print('Test passed with errors')

Quantity of matched mean bills where the popular product is: 2
Quantity of mismatched mean bills where the popular product is: 0
Test passed without errors
